<a href="https://colab.research.google.com/github/Azizkhaled/NLP_with_Aziz/blob/main/Projects/TrainingPretrainedBert/Training_NSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NSP Training

In [ ]:
pip install transformers accelerate -U


In [2]:
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

### Load data, clean, prepare for NSP, and Tokenize

We will utilize the UN report titled "The Question of Palestine" for this project. The data can be accessed using the following link:

https://raw.githubusercontent.com/Azizkhaled/NLP/main/Data/UN_text.txt

We'll start by loading the data from the provided URL and removing any duplicate entries while preserving the order of appearance.

In [3]:
import requests

# function to remove duplicates while keeping the same order
def remove_duplicates_keep_order(input_list):
    seen = set()
    result = []

    for item in input_list:
        if item.strip() != '' and item not in seen:  # Check if the line is not empty and not seen before
            result.append(item)
            seen.add(item)

    return result


In [4]:
data = requests.get('https://raw.githubusercontent.com/Azizkhaled/NLP/main/Data/UN_text.txt')
text = data.text.split('\n')

text = remove_duplicates_keep_order(text)
print( len(text),': ', text[0:3])

886 :  ['The question of Palestine was brought before the United Nations shortly after the end of the Second World War.\r', 'The origins of the Palestine problem as an international issue, however, lie in events occurring towards the end of the First World War. These events led to a League of Nations decision to place Palestine under the administration of Great Britain as the Mandatory Power under the Mandates System adopted by the League. In principle, the Mandate was meant to be in the nature of a transitory phase until Palestine attained the status of a fully independent nation, a status provisionally recognized in the League’s Covenant, but in fact the Mandate’s historical evolution did not result in the emergence of Palestine as an independent nation.\r', 'The decision on the Mandate did not take into account the wishes of the people of Palestine, despite the Covenant’s requirements that “the wishes of these communities must be a principal consideration in the selection of the Man

In [5]:
text[:3]

['The question of Palestine was brought before the United Nations shortly after the end of the Second World War.\r',
 'The origins of the Palestine problem as an international issue, however, lie in events occurring towards the end of the First World War. These events led to a League of Nations decision to place Palestine under the administration of Great Britain as the Mandatory Power under the Mandates System adopted by the League. In principle, the Mandate was meant to be in the nature of a transitory phase until Palestine attained the status of a fully independent nation, a status provisionally recognized in the League’s Covenant, but in fact the Mandate’s historical evolution did not result in the emergence of Palestine as an independent nation.\r',
 'The decision on the Mandate did not take into account the wishes of the people of Palestine, despite the Covenant’s requirements that “the wishes of these communities must be a principal consideration in the selection of the Mandator

In [7]:
text[2].split('.')

['The decision on the Mandate did not take into account the wishes of the people of Palestine, despite the Covenant’s requirements that “the wishes of these communities must be a principal consideration in the selection of the Mandatory”',
 ' This assumed special significance because, almost five years before receiving the mandate from the League of Nations, the British Government had given commitments to the Zionist Organization regarding the establishment of a Jewish national home in Palestine, for which Zionist leaders had pressed a claim of “historical connection” since their ancestors had lived in Palestine two thousand years earlier before dispersing in the “Diaspora”',
 '\r']

We'll assign a 50% probability of using the genuine next sentence, and 50% probability of using another random sentence.

In [28]:
bag = [item for sentence in text for item in sentence.split('.') if item != '\r' and len(item)>20]  # remove empty and very short sentecnes
bag_size = len(bag)

# this bag will be used to pull from later on

In [27]:
bag_size

1434

#### Create our 50/50 NSP training data.

In [35]:
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    sentences = [
        sentence for sentence in paragraph.split('.') if sentence != '\r' and len(sentence)>20
    ]
    num_sentences = len(sentences)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            label.append(1)

In [36]:
for i in range(3):
    print(label[i])
    print(sentence_a[i] + '\n---')
    print(sentence_b[i] + '\n')

1
 These events led to a League of Nations decision to place Palestine under the administration of Great Britain as the Mandatory Power under the Mandates System adopted by the League
---
“(c) That the Jewish Agency be vested with the control of immigration into Palestine and with the necessary authority for the upbuilding of the country”

1
The decision on the Mandate did not take into account the wishes of the people of Palestine, despite the Covenant’s requirements that “the wishes of these communities must be a principal consideration in the selection of the Mandatory”
---
110 Translated from Pic, Pierre, “Le Régime du Mandat d’après le Traité de Versailles”: Revue générale de Droit International Public, vol

0
 The indigenous people of Palestine, whose forefathers had inhabited the land for virtually the two preceding millennia felt this design to be a violation of their natural and inalienable rights
---
 They also viewed it as an infringement of assurances of independence given 

#### Tokenize the data

In [37]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
inputs.keys()


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

#### Adding the labels tensor

In [39]:
inputs['labels'] = torch.LongTensor([label]).T

In [40]:
inputs.labels[:10]


tensor([[1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1]])

###  Create a PyTorch dataset from our data

In [41]:
class UnDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [42]:
dataset = UnDataset(inputs)

## Training Method 1: Pytorch Training

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

In [44]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)


In [45]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-6)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [46]:
from tqdm import tqdm  # for our progress bar
# we need to indlude token_type_ids

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/21 [00:00<?, ?it/s]<ipython-input-41-b95cca338fe7>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1: 100%|██████████| 21/21 [00:27<00:00,  1.29s/it, loss=0.539]
